# Bay Area Tech Layoffs Analysis (2020-2024)

## Project Overview
This project analyzes tech layoff patterns in the San Francisco Bay Area from 2020-2024 to identify:
- Which companies and industries were hit hardest
- Temporal trends in layoffs
- Company stage correlations
- Geographic patterns within the Bay Area

**Author:** [Your Name]  
**Date:** February 2026  
**Tools:** Python (Pandas, Matplotlib, Seaborn), SQL (SQLite)


## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sqlite3
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_csv('layoffs_data.csv')

# Display basic information
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Check data types and missing values
print("Data Types and Missing Values:")
print(df.info())
print("\nMissing Values Count:")
print(df.isnull().sum())

## 2. Data Cleaning

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# 1. Convert date column to datetime
df_clean['date'] = pd.to_datetime(df_clean['date'])

# 2. Extract year, month, quarter
df_clean['year'] = df_clean['date'].dt.year
df_clean['month'] = df_clean['date'].dt.month
df_clean['quarter'] = df_clean['date'].dt.quarter
df_clean['year_quarter'] = df_clean['year'].astype(str) + '-Q' + df_clean['quarter'].astype(str)

# 3. Clean percentage_laid_off (convert to actual percentage)
df_clean['percentage_laid_off'] = df_clean['percentage_laid_off'] * 100

# 4. Handle missing values
# Fill missing total_laid_off with 0 (these are cases where company didn't report or no layoffs)
df_clean['total_laid_off'].fillna(0, inplace=True)

# 5. Create categorical bins for layoff size
def categorize_layoff_size(num):
    if num == 0:
        return 'No Layoffs'
    elif num < 100:
        return 'Small (<100)'
    elif num < 500:
        return 'Medium (100-500)'
    elif num < 1000:
        return 'Large (500-1000)'
    else:
        return 'Very Large (1000+)'

df_clean['layoff_size_category'] = df_clean['total_laid_off'].apply(categorize_layoff_size)

# 6. Remove duplicates if any
df_clean = df_clean.drop_duplicates()

print(f"Data cleaning complete!")
print(f"Original rows: {len(df)}, Clean rows: {len(df_clean)}")
print(f"\nCleaned dataset preview:")
df_clean.head()

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Summary statistics
print("Summary Statistics for Total Layoffs:")
print(df_clean['total_laid_off'].describe())

print("\nSummary Statistics for Percentage Laid Off:")
print(df_clean['percentage_laid_off'].describe())

In [ ]:
# Key Findings

# 1. Total layoffs in dataset
total_layoffs = df_clean['total_laid_off'].sum()
total_companies = df_clean['company'].nunique()
avg_layoffs_per_company = total_layoffs / total_companies

print("=" * 60)
print("KEY FINDINGS")
print("=" * 60)
print(f"Total Layoffs in Bay Area (2020-2024): {total_layoffs:,.0f}")
print(f"Number of Companies with Layoffs: {total_companies}")
print(f"Average Layoffs per Company: {avg_layoffs_per_company:,.0f}")
print("=" * 60)

## 4. Analysis by Industry

In [ ]:
# Layoffs by industry
industry_layoffs = df_clean.groupby('industry').agg({
    'total_laid_off': 'sum',
    'company': 'count'
}).rename(columns={'company': 'num_companies'}).sort_values('total_laid_off', ascending=False)

industry_layoffs['avg_per_company'] = industry_layoffs['total_laid_off'] / industry_layoffs['num_companies']

print("Layoffs by Industry:")
print(industry_layoffs.head(10))

## 5. Analysis by Company

In [ ]:
# Top 15 companies by total layoffs
top_companies = df_clean.groupby('company')['total_laid_off'].sum().sort_values(ascending=False).head(15)

print("Top 15 Companies by Total Layoffs:")
print(top_companies)

## 6. Temporal Analysis

In [ ]:
# Layoffs by year
yearly_layoffs = df_clean.groupby('year')['total_laid_off'].sum().sort_index()

print("Total Layoffs by Year:")
print(yearly_layoffs)

# Layoffs by quarter
quarterly_layoffs = df_clean.groupby('year_quarter')['total_laid_off'].sum().sort_index()

print("\nTop 10 Quarters by Layoffs:")
print(quarterly_layoffs.sort_values(ascending=False).head(10))

## 7. Analysis by Company Stage

In [ ]:
# Layoffs by company stage
stage_layoffs = df_clean.groupby('stage').agg({
    'total_laid_off': 'sum',
    'company': 'count'
}).rename(columns={'company': 'num_companies'}).sort_values('total_laid_off', ascending=False)

print("Layoffs by Company Stage:")
print(stage_layoffs)

## 8. Geographic Analysis

In [ ]:
# Layoffs by Bay Area city
location_layoffs = df_clean.groupby('location').agg({
    'total_laid_off': 'sum',
    'company': 'count'
}).rename(columns={'company': 'num_companies'}).sort_values('total_laid_off', ascending=False)

print("Layoffs by Bay Area Location:")
print(location_layoffs.head(10))

## 9. SQL Analysis

Let's create a SQLite database and run some SQL queries for analysis

In [ ]:
# Create SQLite database
conn = sqlite3.connect('bay_area_layoffs.db')

# Write dataframe to SQL
df_clean.to_sql('layoffs', conn, if_exists='replace', index=False)

print("Database created successfully!")

In [ ]:
# SQL Query 1: Top 10 companies by total layoffs
query1 = """
SELECT 
    company,
    location,
    industry,
    SUM(total_laid_off) as total_layoffs,
    ROUND(AVG(percentage_laid_off), 2) as avg_percentage
FROM layoffs
GROUP BY company, location, industry
HAVING total_layoffs > 0
ORDER BY total_layoffs DESC
LIMIT 10;
"""

result1 = pd.read_sql_query(query1, conn)
print("Top 10 Companies by Total Layoffs (SQL Query):")
print(result1)

In [ ]:
# SQL Query 2: Industry analysis with year-over-year comparison
query2 = """
SELECT 
    industry,
    year,
    SUM(total_laid_off) as total_layoffs,
    COUNT(DISTINCT company) as num_companies
FROM layoffs
WHERE total_laid_off > 0
GROUP BY industry, year
ORDER BY industry, year;
"""

result2 = pd.read_sql_query(query2, conn)
print("Industry Analysis by Year (SQL Query):")
print(result2.head(20))

In [ ]:
# SQL Query 3: Companies that laid off highest percentage of workforce
query3 = """
SELECT 
    company,
    location,
    industry,
    total_laid_off,
    ROUND(percentage_laid_off, 2) as pct_laid_off,
    stage,
    date
FROM layoffs
WHERE percentage_laid_off >= 30
ORDER BY percentage_laid_off DESC
LIMIT 15;
"""

result3 = pd.read_sql_query(query3, conn)
print("Companies with Highest Percentage Laid Off (>= 30%):")
print(result3)

## 10. Data Visualizations

In [ ]:
# Visualization 1: Top 10 Industries by Total Layoffs
plt.figure(figsize=(12, 6))
top_industries = industry_layoffs.head(10)
plt.barh(top_industries.index, top_industries['total_laid_off'], color='steelblue')
plt.xlabel('Total Layoffs', fontsize=12, fontweight='bold')
plt.ylabel('Industry', fontsize=12, fontweight='bold')
plt.title('Top 10 Industries by Total Layoffs (Bay Area 2020-2024)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
for i, v in enumerate(top_industries['total_laid_off']):
    plt.text(v + 100, i, f'{v:,.0f}', va='center', fontsize=10)
plt.tight_layout()
plt.savefig('industry_layoffs.png', dpi=300, bbox_inches='tight')
plt.show()

print("Chart saved as 'industry_layoffs.png'")

In [ ]:
# Visualization 2: Layoffs Timeline (Monthly Trend)
plt.figure(figsize=(14, 6))
monthly_data = df_clean[df_clean['total_laid_off'] > 0].groupby(df_clean['date'].dt.to_period('M'))['total_laid_off'].sum()
monthly_data_df = pd.DataFrame({'month': monthly_data.index.to_timestamp(), 'layoffs': monthly_data.values})

plt.plot(monthly_data_df['month'], monthly_data_df['layoffs'], marker='o', linewidth=2, color='darkred')
plt.fill_between(monthly_data_df['month'], monthly_data_df['layoffs'], alpha=0.3, color='lightcoral')
plt.xlabel('Date', fontsize=12, fontweight='bold')
plt.ylabel('Total Layoffs', fontsize=12, fontweight='bold')
plt.title('Bay Area Tech Layoffs Over Time (Monthly)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('layoffs_timeline.png', dpi=300, bbox_inches='tight')
plt.show()

print("Chart saved as 'layoffs_timeline.png'")

In [ ]:
# Visualization 3: Top 15 Companies by Layoffs
plt.figure(figsize=(12, 8))
top_15_companies = df_clean.groupby('company')['total_laid_off'].sum().sort_values(ascending=False).head(15)
colors = plt.cm.Spectral(np.linspace(0, 1, 15))
plt.barh(range(len(top_15_companies)), top_15_companies.values, color=colors)
plt.yticks(range(len(top_15_companies)), top_15_companies.index)
plt.xlabel('Total Layoffs', fontsize=12, fontweight='bold')
plt.ylabel('Company', fontsize=12, fontweight='bold')
plt.title('Top 15 Bay Area Companies by Total Layoffs', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
for i, v in enumerate(top_15_companies.values):
    plt.text(v + 100, i, f'{v:,.0f}', va='center', fontsize=9)
plt.tight_layout()
plt.savefig('top_companies_layoffs.png', dpi=300, bbox_inches='tight')
plt.show()

print("Chart saved as 'top_companies_layoffs.png'")

In [ ]:
# Visualization 4: Layoffs by Company Stage (Pie Chart)
plt.figure(figsize=(10, 8))
stage_data = df_clean[df_clean['total_laid_off'] > 0].groupby('stage')['total_laid_off'].sum().sort_values(ascending=False)
colors_stage = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0', '#ffb3e6']
explode = [0.05 if i == 0 else 0 for i in range(len(stage_data))]

plt.pie(stage_data.values, labels=stage_data.index, autopct='%1.1f%%', startangle=90,
        colors=colors_stage, explode=explode, shadow=True)
plt.title('Distribution of Layoffs by Company Stage', fontsize=14, fontweight='bold')
plt.axis('equal')
plt.tight_layout()
plt.savefig('layoffs_by_stage.png', dpi=300, bbox_inches='tight')
plt.show()

print("Chart saved as 'layoffs_by_stage.png'")

In [ ]:
# Visualization 5: Layoffs by Location (Top 10 Bay Area Cities)
plt.figure(figsize=(12, 6))
top_locations = location_layoffs.head(10)
plt.bar(range(len(top_locations)), top_locations['total_laid_off'], color='teal', alpha=0.7)
plt.xticks(range(len(top_locations)), top_locations.index, rotation=45, ha='right')
plt.xlabel('Location', fontsize=12, fontweight='bold')
plt.ylabel('Total Layoffs', fontsize=12, fontweight='bold')
plt.title('Top 10 Bay Area Locations by Layoffs', fontsize=14, fontweight='bold')
for i, v in enumerate(top_locations['total_laid_off']):
    plt.text(i, v + 200, f'{v:,.0f}', ha='center', fontsize=9, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('layoffs_by_location.png', dpi=300, bbox_inches='tight')
plt.show()

print("Chart saved as 'layoffs_by_location.png'")

In [ ]:
# Visualization 6: Heatmap - Layoffs by Industry and Year
plt.figure(figsize=(14, 8))
heatmap_data = df_clean[df_clean['total_laid_off'] > 0].pivot_table(
    values='total_laid_off', 
    index='industry', 
    columns='year', 
    aggfunc='sum', 
    fill_value=0
)

sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='YlOrRd', linewidths=0.5, cbar_kws={'label': 'Total Layoffs'})
plt.title('Layoffs Heatmap: Industry vs Year', fontsize=14, fontweight='bold')
plt.xlabel('Year', fontsize=12, fontweight='bold')
plt.ylabel('Industry', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig('layoffs_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("Chart saved as 'layoffs_heatmap.png'")

## 11. Key Insights and Conclusions

In [ ]:
# Generate insights
print("=" * 80)
print("KEY INSIGHTS FROM ANALYSIS")
print("=" * 80)

# Insight 1: Worst year
worst_year = yearly_layoffs.idxmax()
worst_year_layoffs = yearly_layoffs.max()
print(f"\n1. WORST YEAR FOR LAYOFFS:")
print(f"   {worst_year} had {worst_year_layoffs:,.0f} layoffs in the Bay Area")

# Insight 2: Hardest hit industry
top_industry = industry_layoffs.index[0]
top_industry_layoffs = industry_layoffs.iloc[0]['total_laid_off']
print(f"\n2. HARDEST HIT INDUSTRY:")
print(f"   {top_industry} experienced {top_industry_layoffs:,.0f} layoffs")

# Insight 3: Company stage most affected
top_stage = stage_layoffs.index[0]
top_stage_layoffs = stage_layoffs.iloc[0]['total_laid_off']
print(f"\n3. MOST AFFECTED COMPANY STAGE:")
print(f"   {top_stage} companies had {top_stage_layoffs:,.0f} layoffs")

# Insight 4: Bay Area epicenter
top_city = location_layoffs.index[0]
top_city_layoffs = location_layoffs.iloc[0]['total_laid_off']
print(f"\n4. BAY AREA EPICENTER:")
print(f"   {top_city} led with {top_city_layoffs:,.0f} layoffs")

# Insight 5: Companies with 100% layoffs (shut down)
shutdown_companies = df_clean[df_clean['percentage_laid_off'] == 100]['company'].unique()
print(f"\n5. COMPANIES THAT SHUT DOWN (100% layoffs):")
print(f"   {len(shutdown_companies)} companies completely shut down")
print(f"   Examples: {', '.join(shutdown_companies[:5])}")

print("\n" + "=" * 80)

In [ ]:
# Close database connection
conn.close()
print("\nAnalysis complete! All visualizations saved.")